In [11]:
print('hello world')

hello world


In [16]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import time

# Step 1: Get S&P 500 companies table
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
headers = {"User-Agent": "Mozilla/5.0 (compatible; sp500-fetcher/1.0; +https://example.com)"}
resp = requests.get(url, headers=headers, timeout=20)
resp.raise_for_status()

soup = BeautifulSoup(resp.text, "html.parser")
table = soup.find("table", {"class": "wikitable"})

df = pd.read_html(str(table))[0]

# Step 2: Get company Wikipedia URLs
base = "https://en.wikipedia.org"
links = [base + a["href"] for a in table.select("tbody tr td:nth-of-type(2) a[href]")]
df = df.iloc[:len(links)].copy()
df["Wikipedia_URL"] = links

# Step 3: Fetch short intro for each company
def get_intro(url):
    title = url.split("/wiki/")[-1]
    api_url = f"https://en.wikipedia.org/api/rest_v1/page/summary/{title}"
    try:
        r = requests.get(api_url, headers=headers, timeout=10)
        r.raise_for_status()
        return r.json().get("extract", "")
    except Exception:
        return ""

intros = []
for link in tqdm(df["Wikipedia_URL"], desc="Fetching intros"):
    intros.append(get_intro(link))
    time.sleep(0.3)  # delay to avoid being blocked

# Step 4: Keep only required columns and add ID
df = df[["Symbol", "Security", "Wikipedia_URL"]].copy()
df["Intro"] = intros
df.insert(0, "ID", range(1, len(df) + 1))

# Step 5: Save to CSV
df.to_csv("sp500_wiki_intros.csv", index=False)
print("✅ Exported", len(df), "rows to sp500_wiki_intros.csv")


C:\Users\jacks\AppData\Local\Temp\ipykernel_6792\3343189255.py:16: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(str(table))[0]
Fetching intros: 100%|██████████| 503/503 [03:32<00:00,  2.37it/s]

✅ Exported 503 rows to sp500_wiki_intros.csv


In [18]:
import csv
import pandas as pd

# Drop existing ID column if it already exists
if "ID" in df.columns:
    df = df.drop(columns=["ID"])

# 1) Basic cleaning
df["Symbol"] = df["Symbol"].astype(str).str.strip()
df["Security"] = df["Security"].astype(str).str.strip()
df = df[df["Symbol"] != ""]
df = df.dropna(subset=["Symbol"])

# 2) Deduplicate by ticker symbol
before = len(df)
df = df.drop_duplicates(subset=["Symbol"], keep="first").reset_index(drop=True)
after = len(df)

print(f"Rows before dedupe: {before}, after dedupe: {after} (unique symbols)")

# 3) Recreate ID starting at 1 and keep only requested columns
df.insert(0, "ID", range(1, len(df) + 1))
df = df[["ID", "Symbol", "Security", "Wikipedia_URL", "Intro"]]

# 4) Export safely (quotes protect commas in text)
out_path = "sp500_wiki_intros_clean.csv"
df.to_csv(out_path, index=False, encoding="utf-8", quoting=csv.QUOTE_ALL)

print(f"✅ Cleaned and saved {len(df)} rows to {out_path}")
print("Unique symbols:", df['Symbol'].nunique())


Rows before dedupe: 503, after dedupe: 503 (unique symbols)
✅ Cleaned and saved 503 rows to sp500_wiki_intros_clean.csv
Unique symbols: 503


In [19]:
# %% Step 1: prepare text, create embeddings, and store them with an index
# pip install openai numpy pandas tqdm

import os
import time
import math
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
from openai import OpenAI

# -------- CONFIG --------
CSV_CANDIDATES = ["sp500_wiki_intros_clean.csv", "sp500_wiki_intros.csv", "sp500_wiki_intros_cleaned.csv"]
EMB_DIR = "embeddings"
EMB_MATRIX_PATH = os.path.join(EMB_DIR, "embeddings.npy")
EMB_INDEX_CSV = os.path.join(EMB_DIR, "embeddings_index.csv")
MODEL = "text-embedding-3-small"   # cost-effective default (1536d)
BATCH_SIZE = 16
SLEEP_BETWEEN_BATCHES = 0.15
MAX_RETRIES = 5
NORMALIZE = True   # L2-normalize embeddings (recommended for cosine similarity)
# ------------------------

os.makedirs(EMB_DIR, exist_ok=True)

# 1) Load CSV (try common names) ------------------------------------------------
csv_path = None
for p in CSV_CANDIDATES:
    if os.path.exists(p):
        csv_path = p
        break
if csv_path is None:
    raise FileNotFoundError(
        "Couldn't find your CSV. Make sure your cleaned CSV (sp500_wiki_intros_clean.csv or similar) is in the notebook working directory."
    )

df = pd.read_csv(csv_path, encoding="utf-8")
required_cols = {"ID", "Symbol", "Security", "Wikipedia_URL", "Intro"}
if not required_cols.issubset(set(df.columns)):
    raise RuntimeError(f"CSV must include columns: {required_cols}. Found: {list(df.columns)}")

# remove rows with empty intro (or set fallback)
df["Intro"] = df["Intro"].fillna("").astype(str)
empty_count = (df["Intro"].str.strip() == "").sum()
if empty_count > 0:
    print(f"Warning: {empty_count} rows have empty Intro text. They will still be embedded (but may be low-information).")

N = len(df)
print(f"Loaded {N} rows from {csv_path}")

# 2) Setup OpenAI client -------------------------------------------------------
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise RuntimeError("OPENAI_API_KEY environment variable not set. Set it securely before running.")
client = OpenAI()

# 3) Helper: call embeddings with retries --------------------------------------
def embed_batch(texts, model=MODEL, max_retries=MAX_RETRIES):
    attempt = 0
    backoff = 1.0
    while attempt <= max_retries:
        try:
            resp = client.embeddings.create(model=model, input=texts)
            vectors = [item.embedding for item in resp.data]
            return vectors
        except Exception as e:
            attempt += 1
            if attempt > max_retries:
                raise
            wait = backoff * (2 ** (attempt - 1))
            print(f"Embedding batch failed (attempt {attempt}/{max_retries}). Retrying in {wait:.1f}s. Error: {e}")
            time.sleep(wait)
    raise RuntimeError("Unreachable: embed_batch failed after retries")

# 4) Batch embedding loop -----------------------------------------------------
all_vectors = []
index_rows = []   # metadata: vector_index -> csv row
texts = df["Intro"].tolist()

for i in tqdm(range(0, N, BATCH_SIZE), desc="Embedding batches"):
    batch_indices = list(range(i, min(i + BATCH_SIZE, N)))
    batch_texts = [texts[idx] if texts[idx].strip() != "" else " " for idx in batch_indices]
    vectors = embed_batch(batch_texts, model=MODEL)
    # ensure vectors length matches input
    if len(vectors) != len(batch_texts):
        raise RuntimeError("Embedding API returned unexpected number of vectors.")
    # append and store mapping
    for j, vec in zip(batch_indices, vectors):
        all_vectors.append(np.array(vec, dtype=np.float32))
        index_rows.append({
            "vector_index": len(all_vectors)-1,
            "ID": int(df.loc[j, "ID"]),
            "Symbol": df.loc[j, "Symbol"],
            "Security": df.loc[j, "Security"],
            "Wikipedia_URL": df.loc[j, "Wikipedia_URL"],
        })
    time.sleep(SLEEP_BETWEEN_BATCHES)

# 5) Build embeddings matrix & normalize (optional) ----------------------------
if not all_vectors:
    raise RuntimeError("No embeddings produced.")

emb_matrix = np.stack(all_vectors)   # shape (N, D)
print("Embeddings matrix shape:", emb_matrix.shape)

if NORMALIZE:
    norms = np.linalg.norm(emb_matrix, axis=1, keepdims=True)
    norms[norms == 0.0] = 1.0
    emb_matrix = emb_matrix / norms
    print("L2-normalized embeddings (for cosine similarity).")

# 6) Persist matrix + index ---------------------------------------------------
np.save(EMB_MATRIX_PATH, emb_matrix)
print("Saved embeddings matrix to", EMB_MATRIX_PATH)

# write index CSV (maps vector_index -> company metadata)
with open(EMB_INDEX_CSV, "w", newline="", encoding="utf-8") as f:
    writer = csv.DictWriter(f, fieldnames=["vector_index", "ID", "Symbol", "Security", "Wikipedia_URL"])
    writer.writeheader()
    for r in index_rows:
        writer.writerow(r)
print("Saved embeddings index to", EMB_INDEX_CSV)

# 7) Basic sanity checks ------------------------------------------------------
if emb_matrix.shape[0] != len(index_rows) or emb_matrix.shape[0] != N:
    print("Warning: counts mismatch. embedding rows:", emb_matrix.shape[0], "index rows:", len(index_rows), "CSV rows:", N)
else:
    print("All counts consistent. Ready for clustering/visualization.")

# Quick example: load back to verify
# loaded = np.load(EMB_MATRIX_PATH)
# print('loaded shape', loaded.shape)


Loaded 503 rows from sp500_wiki_intros_clean.csv


Embedding batches: 100%|██████████| 32/32 [00:42<00:00,  1.34s/it]

Embeddings matrix shape: (503, 1536)
L2-normalized embeddings (for cosine similarity).
Saved embeddings matrix to embeddings\embeddings.npy
Saved embeddings index to embeddings\embeddings_index.csv
All counts consistent. Ready for clustering/visualization.
